Now we will look at replacing some of the numerical values that are missing, using WOE and median/mean

In [10]:
import pandas as pd
import numpy as np

In [11]:
data = pd.read_csv('Test.csv')

C:\Users\emmak\AppData\Local\Temp\ipykernel_19504\2780563573.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Test.csv')


In [12]:
#renaming columns so easier to understand
data.rename(columns={
    'fecha_dato': 'date',
    'ncodpers': 'customer code',
    'ind_empleado': 'employee index',
    'pais_residencia': 'country',
    'sexo': 'sex',
    'fecha_alta': 'contract date',
    'ind_nuevo': 'customer index',
    'antiguedad': 'customer seniority',
}, inplace=True)

In [32]:
#look for outliers
data.info()   #look at data frame
data.describe()   #have a look at mean, medians and quartiles to see if the data is skewed or if there are any outliers

#from .describe() we can see that there appears to be an outlier in age as the maximum age is 164 which can't be right

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 929615 entries, 0 to 929614
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   date                   929615 non-null  object 
 1   customer code          929615 non-null  int64  
 2   employee index         929615 non-null  object 
 3   country                929615 non-null  object 
 4   sex                    929610 non-null  object 
 5   age                    929615 non-null  int64  
 6   contract date          929615 non-null  object 
 7   customer index         929615 non-null  int64  
 8   customer seniority     929615 non-null  int64  
 9   indrel                 929615 non-null  int64  
 10  ult_fec_cli_1t         1683 non-null    object 
 11  indrel_1mes            929592 non-null  float64
 12  tiprel_1mes            929592 non-null  object 
 13  indresi                929615 non-null  object 
 14  indext                 929615 non-nu

,customer code,age,customer index,customer seniority,indrel,indrel_1mes,tipodom,cod_prov,ind_actividad_cliente
count,9.296150e+05,929615.000000,929615.0,929615.000000,929615.000000,929592.000000,929615.0,925619.000000,929615.000000
mean,8.794566e+05,40.249591,0.0,77.728413,1.177422,1.000058,1.0,26.546150,0.425272
std,4.481569e+05,17.184405,0.0,1797.824518,4.166039,0.010779,0.0,12.836614,0.494384
min,1.588900e+04,2.000000,0.0,-999999.000000,1.000000,1.000000,1.0,1.000000,0.000000
25%,4.833615e+05,25.000000,0.0,23.000000,1.000000,1.000000,1.0,15.000000,0.000000
50%,9.664250e+05,39.000000,0.0,55.000000,1.000000,1.000000,1.0,28.000000,0.000000
75%,1.264316e+06,51.000000,0.0,136.000000,1.000000,1.000000,1.0,35.000000,1.000000
max,1.553689e+06,118.000000,0.0,257.000000,99.000000,3.000000,1.0,52.000000,1.000000


In [14]:
# for age use median imputation

valid_values = range(0,120)   #age range, set limit to 120 as likely no one older than that

median_value = data['age'].median()     #using median as less sensitive to outliers than mean

for i, value in enumerate(data['age']):
    if value not in valid_values:
        data.at[i, 'indfall'] = 'N'   #mistakenly put as not deceased
        data.at[i, 'age'] = median_value  #for unknown

In [24]:
#for customer seniority again use median imputation

median_value = data['customer index'].median() 
standard_deviation = np.std(data['customer index'])

valid_values = range(int(median_value -3*standard_deviation), int(median_value + 3*standard_deviation) )

for i, value in enumerate(data['customer index']):
    if value not in valid_values:
        data.at[i, 'customer index'] = median_value  #for unknown

In [25]:
#for cod_prov use multiple imputation

pip install miceforest

In [27]:
from miceforest import ImputationKernel

In [ ]:
mice_kernel = ImputationKernel(
data['cod_prov'],
save_all_iterations = True,
random_state = 2023
)

In [ ]:
mice_kernel.mice(2)
mice_imputation = mice_kernel.complete_data()
mice_imputation.head()

In [35]:
#looking at WOE for deceased

goods = 0             #good if employee not dead
valid_values = ['S']
for i, value in enumerate(data['employee index']):
    if value not in valid_values:
        goods +=1

bads = 0                 #bad if employee dead
valid_values = ['N']
for i, value in enumerate(data['employee index']):
    if value not in valid_values:
        bads +=1      

In [40]:
WOE = np.log(bads/goods)

In [41]:
WOE    #WOE is -7.5 for deceased vs non-deceased 

-7.490620842041455

In [44]:
percent_good = 100*goods/(goods+bads)
percent_bad = 100*bads/(goods+bads)

i_v = (percent_bad - percent_good)*WOE

In [45]:
i_v

#information value is 748,suspicious relationship

748.226153812036